# Week 6 Minor Assignment: Interactive Visualization Conversion

## 🎯 Assignment Overview

**Due Date**: End of Wednesday class session  
**Points**: 20 points  
**Individual Assignment**

## 📚 Learning Objectives

This assignment reinforces **Wednesday's interactive visualization topics**:
- Converting static matplotlib/seaborn charts to interactive Plotly visualizations
- Creating engaging dashboard-style layouts
- Implementing interactive features (hover, zoom, filters)
- Using real business data for meaningful insights

## 🏢 Business Context

You work as a data analyst for Olist, Brazil's largest e-commerce marketplace. The marketing team has been using static charts in their presentations, but they want to upgrade to interactive visualizations that executives can explore during meetings.

Your task is to convert existing static analysis into an interactive dashboard that tells a compelling story about Olist's business performance.

## 📊 Data Source

You'll connect to the Olist dataset via **Supabase PostgreSQL** to access real Brazilian e-commerce data:
- 100K+ customer orders
- Geographic data across Brazilian states
- Product categories and sales performance
- Customer satisfaction scores

In [ ]:
# Setup and Data Loading
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.offline as pyo
import sqlalchemy
from sqlalchemy import create_engine, text
import warnings
import os
from dotenv import load_dotenv

warnings.filterwarnings('ignore')
pyo.init_notebook_mode(connected=True)

# Student Information
STUDENT_INFO = {
    'name': 'YOUR_FULL_NAME',
    'student_id': 'YOUR_STUDENT_ID',
    'submission_date': 'YYYY-MM-DD'
}

print(f"📊 Interactive Visualization Assignment")
print(f"👤 Student: {STUDENT_INFO['name']}")
print(f"🆔 ID: {STUDENT_INFO['student_id']}")

## Database Connection Setup

**⚠️ Important**: Create a `.env` file in your project directory with your Supabase credentials:

```
POSTGRES_HOST=your_supabase_host
POSTGRES_PORT=6543
POSTGRES_DATABASE=postgres
POSTGRES_USER=your_username
POSTGRES_PASSWORD=your_password
```

In [ ]:
# Database Connection Setup
load_dotenv()

# Supabase PostgreSQL Configuration
DATABASE_CONFIG = {
    'host': os.getenv('POSTGRES_HOST'),
    'port': int(os.getenv('POSTGRES_PORT', 5432)),
    'database': os.getenv('POSTGRES_DATABASE'),
    'user': os.getenv('POSTGRES_USER'),
    'password': os.getenv('POSTGRES_PASSWORD')
}

# Verify configuration
if not all([DATABASE_CONFIG['host'], DATABASE_CONFIG['user'], DATABASE_CONFIG['password']]):
    print("❌ Missing database credentials. Please check your .env file.")
    print("📝 Required variables: POSTGRES_HOST, POSTGRES_USER, POSTGRES_PASSWORD")
else:
    print("✅ Database configuration loaded successfully")

# Create database connection
POSTGRES_URL = f"postgresql://{DATABASE_CONFIG['user']}:{DATABASE_CONFIG['password']}@{DATABASE_CONFIG['host']}:{DATABASE_CONFIG['port']}/{DATABASE_CONFIG['database']}"

def create_db_engine():
    """Create database engine with error handling."""
    try:
        engine = create_engine(POSTGRES_URL, pool_size=3, max_overflow=5)
        # Test connection
        with engine.connect() as conn:
            conn.execute(text("SELECT 1"))
        print("🔗 Successfully connected to Supabase PostgreSQL")
        return engine
    except Exception as e:
        print(f"❌ Database connection failed: {e}")
        return None

# Create database engine
db_engine = create_db_engine()

## Task 1: Geographic Sales Analysis (5 points)

**Business Question**: "Which Brazilian states are our top markets, and how can we visualize this for executive presentations?"

Convert a static bar chart to an interactive choropleth map showing sales by Brazilian state.

In [ ]:
# Task 1: Load geographic sales data
def load_geographic_sales_data(engine):
    """
    Load sales data by Brazilian state from Supabase.
    """
    if engine is None:
        print("❌ No database connection available")
        return pd.DataFrame()
    
    query = """
    SELECT 
        c.customer_state,
        COUNT(DISTINCT o.order_id) as total_orders,
        COUNT(DISTINCT c.customer_id) as unique_customers,
        SUM(oi.price::numeric + oi.freight_value::numeric) as total_revenue,
        AVG(oi.price::numeric) as avg_order_value
    FROM "olist_sales_data_set"."olist_customers_dataset" c
    INNER JOIN "olist_sales_data_set"."olist_orders_dataset" o ON c.customer_id = o.customer_id
    INNER JOIN "olist_sales_data_set"."olist_order_items_dataset" oi ON o.order_id = oi.order_id
    WHERE c.customer_state IS NOT NULL 
        AND o.order_status = 'delivered'
        AND oi.price IS NOT NULL
    GROUP BY c.customer_state
    ORDER BY total_revenue DESC
    """
    
    try:
        df = pd.read_sql(text(query), engine)
        print(f"✅ Loaded {len(df)} Brazilian states with sales data")
        return df
    except Exception as e:
        print(f"❌ Failed to load geographic data: {e}")
        return pd.DataFrame()

# Load the data
geo_sales_data = load_geographic_sales_data(db_engine)
if not geo_sales_data.empty:
    display(geo_sales_data.head())

In [ ]:
# Task 1.1: Create STATIC visualization (what we want to replace)
if not geo_sales_data.empty:
    plt.figure(figsize=(12, 8))
    sns.barplot(data=geo_sales_data.head(10), x='total_revenue', y='customer_state', palette='viridis')
    plt.title('Top 10 Brazilian States by Revenue (Static Chart)', fontsize=16, fontweight='bold')
    plt.xlabel('Total Revenue (R$)', fontsize=12)
    plt.ylabel('State', fontsize=12)
    plt.ticklabel_format(style='plain', axis='x')
    plt.tight_layout()
    plt.show()
    
    print("📊 This static chart is informative but lacks interactivity...")
    print("🎯 Let's convert it to an interactive Plotly visualization!")

In [ ]:
# Task 1.2: Create INTERACTIVE Plotly visualization
# TODO: Convert the static bar chart above into an interactive Plotly chart
# Requirements:
# 1. Use plotly.express or plotly.graph_objects
# 2. Add hover information showing all metrics (orders, customers, revenue, AOV)
# 3. Make the chart interactive with zoom and pan capabilities
# 4. Include proper titles and formatting
# 5. Use a professional color scheme

if not geo_sales_data.empty:
    # Your interactive Plotly code here
    fig = px.bar(
        # TODO: Complete this implementation
        # Hint: Use geo_sales_data.head(15) for top 15 states
        # Add hover_data parameter for additional information
        pass
    )
    
    # TODO: Update layout with titles, labels, and styling
    fig.update_layout(
        # Add your layout updates here
        pass
    )
    
    # TODO: Show the figure
    # fig.show()

## Task 2: Customer Satisfaction Trends (5 points)

**Business Question**: "How has customer satisfaction evolved over time, and what insights can we provide to improve the customer experience?"

Create an interactive time series showing customer satisfaction trends with the ability to filter by review scores.

In [ ]:
# Task 2: Load customer satisfaction data over time
def load_satisfaction_trends(engine):
    """
    Load customer satisfaction trends by month.
    """
    if engine is None:
        return pd.DataFrame()
    
    query = """
    SELECT 
        DATE_TRUNC('month', r.review_creation_date) as review_month,
        COUNT(*) as total_reviews,
        AVG(r.review_score::numeric) as avg_satisfaction,
        COUNT(CASE WHEN r.review_score >= 4 THEN 1 END) as positive_reviews,
        COUNT(CASE WHEN r.review_score <= 2 THEN 1 END) as negative_reviews,
        ROUND(COUNT(CASE WHEN r.review_score >= 4 THEN 1 END) * 100.0 / COUNT(*), 1) as satisfaction_rate
    FROM "olist_sales_data_set"."olist_order_reviews_dataset" r
    WHERE r.review_creation_date IS NOT NULL
        AND r.review_score IS NOT NULL
        AND r.review_creation_date >= '2017-01-01'
        AND r.review_creation_date < '2019-01-01'
    GROUP BY DATE_TRUNC('month', r.review_creation_date)
    ORDER BY review_month
    """
    
    try:
        df = pd.read_sql(text(query), engine)
        df['review_month'] = pd.to_datetime(df['review_month'])
        print(f"✅ Loaded satisfaction trends for {len(df)} months")
        return df
    except Exception as e:
        print(f"❌ Failed to load satisfaction data: {e}")
        return pd.DataFrame()

satisfaction_data = load_satisfaction_trends(db_engine)
if not satisfaction_data.empty:
    display(satisfaction_data.head())

In [ ]:
# Task 2.1: Static visualization (baseline)
if not satisfaction_data.empty:
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))
    
    # Average satisfaction over time
    ax1.plot(satisfaction_data['review_month'], satisfaction_data['avg_satisfaction'], 
             marker='o', linewidth=2, markersize=6)
    ax1.set_title('Average Customer Satisfaction Over Time (Static)', fontsize=14, fontweight='bold')
    ax1.set_ylabel('Average Rating (1-5)', fontsize=12)
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(3.5, 4.5)
    
    # Review volume over time
    ax2.bar(satisfaction_data['review_month'], satisfaction_data['total_reviews'], 
            color='steelblue', alpha=0.7)
    ax2.set_title('Review Volume Over Time', fontsize=14, fontweight='bold')
    ax2.set_ylabel('Number of Reviews', fontsize=12)
    ax2.set_xlabel('Month', fontsize=12)
    
    plt.tight_layout()
    plt.show()
    
    print("📊 Static charts show the trends but don't allow exploration...")

In [ ]:
# Task 2.2: Create INTERACTIVE time series with dual axes
# TODO: Create an interactive Plotly chart showing:
# 1. Average satisfaction score over time (line chart)
# 2. Review volume over time (bar chart or area chart)
# 3. Use secondary y-axis for volume
# 4. Add hover information with detailed metrics
# 5. Include range slider for date selection

if not satisfaction_data.empty:
    # TODO: Create subplot with secondary y-axis
    # Hint: Use make_subplots with specs=[{"secondary_y": True}]
    
    # Your code here:
    # fig = make_subplots(...)
    
    # TODO: Add satisfaction trend line
    # fig.add_trace(...)
    
    # TODO: Add review volume bars on secondary y-axis
    # fig.add_trace(..., secondary_y=True)
    
    # TODO: Update layout with titles, axis labels, and range slider
    # fig.update_layout(...)
    
    # TODO: Update y-axes labels
    # fig.update_yaxes(...)
    
    # fig.show()
    pass

## Task 3: Product Category Performance Dashboard (5 points)

**Business Question**: "Which product categories drive the most revenue and have the highest customer satisfaction?"

Create an interactive scatter plot showing the relationship between revenue and satisfaction by product category.

In [ ]:
# Task 3: Load product category performance data
def load_category_performance(engine):
    """
    Load product category performance metrics.
    """
    if engine is None:
        return pd.DataFrame()
    
    query = """
    SELECT 
        p.product_category_name,
        t.product_category_name_english,
        COUNT(DISTINCT oi.order_id) as total_orders,
        SUM(oi.price::numeric) as total_revenue,
        AVG(oi.price::numeric) as avg_price,
        COUNT(DISTINCT p.product_id) as unique_products,
        AVG(r.review_score::numeric) as avg_satisfaction,
        COUNT(r.review_id) as total_reviews
    FROM "olist_sales_data_set"."olist_products_dataset" p
    INNER JOIN "olist_sales_data_set"."olist_order_items_dataset" oi ON p.product_id = oi.product_id
    LEFT JOIN "olist_sales_data_set"."olist_order_reviews_dataset" r ON oi.order_id = r.order_id
    LEFT JOIN "olist_sales_data_set"."product_category_name_translation" t 
        ON p.product_category_name = t.product_category_name
    WHERE p.product_category_name IS NOT NULL
        AND oi.price IS NOT NULL
    GROUP BY p.product_category_name, t.product_category_name_english
    HAVING COUNT(r.review_id) >= 100  -- Only categories with sufficient reviews
    ORDER BY total_revenue DESC
    """
    
    try:
        df = pd.read_sql(text(query), engine)
        # Use English names where available, otherwise use Portuguese
        df['category_display'] = df['product_category_name_english'].fillna(df['product_category_name'])
        print(f"✅ Loaded {len(df)} product categories with performance data")
        return df
    except Exception as e:
        print(f"❌ Failed to load category data: {e}")
        return pd.DataFrame()

category_data = load_category_performance(db_engine)
if not category_data.empty:
    display(category_data.head())

In [ ]:
# Task 3.1: Static scatter plot (baseline)
if not category_data.empty:
    plt.figure(figsize=(12, 8))
    scatter = plt.scatter(category_data['total_revenue'], category_data['avg_satisfaction'], 
                         s=category_data['total_orders']/10, alpha=0.6, c=category_data['avg_price'], 
                         cmap='viridis')
    plt.xlabel('Total Revenue (R$)', fontsize=12)
    plt.ylabel('Average Satisfaction (1-5)', fontsize=12)
    plt.title('Product Category Performance: Revenue vs Satisfaction (Static)', fontsize=14, fontweight='bold')
    plt.colorbar(scatter, label='Average Price (R$)')
    
    # Add annotations for top categories
    for i, row in category_data.head(5).iterrows():
        plt.annotate(row['category_display'][:15], 
                    (row['total_revenue'], row['avg_satisfaction']),
                    xytext=(5, 5), textcoords='offset points', fontsize=8)
    
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    print("📊 Static scatter plot shows relationships but lacks detailed exploration...")

In [ ]:
# Task 3.2: Create INTERACTIVE scatter plot with advanced features
# TODO: Create an interactive Plotly scatter plot with:
# 1. Revenue on x-axis, satisfaction on y-axis
# 2. Bubble size representing total orders
# 3. Color representing average price
# 4. Hover information showing all metrics
# 5. Ability to zoom and select data points
# 6. Professional styling and labels

if not category_data.empty:
    # TODO: Create interactive scatter plot
    # Hint: Use px.scatter with size, color, and hover_data parameters
    
    # Your code here:
    # fig = px.scatter(
    #     category_data,
    #     x='...',
    #     y='...',
    #     size='...',
    #     color='...',
    #     hover_data=[...],
    #     text='...',
    #     title='...'
    # )
    
    # TODO: Update layout and traces
    # fig.update_traces(...)
    # fig.update_layout(...)
    
    # fig.show()
    pass

## Task 4: Executive Dashboard Summary (5 points)

**Business Question**: "Can we create a single interactive dashboard that summarizes our key business metrics for executive presentations?"

Combine multiple visualizations into a cohesive dashboard layout using Plotly subplots.

In [ ]:
# Task 4: Create an executive dashboard with multiple charts
# TODO: Create a dashboard with subplots containing:
# 1. Top 10 states by revenue (bar chart)
# 2. Satisfaction trend over time (line chart)
# 3. Top product categories (horizontal bar chart)
# 4. Revenue vs Satisfaction scatter plot

if not geo_sales_data.empty and not satisfaction_data.empty and not category_data.empty:
    # TODO: Create subplots with 2x2 grid
    # Hint: Use make_subplots with rows=2, cols=2
    
    # Your code here:
    # fig = make_subplots(
    #     rows=2, cols=2,
    #     subplot_titles=('Top States by Revenue', 'Satisfaction Trends', 
    #                     'Top Product Categories', 'Revenue vs Satisfaction'),
    #     specs=[[{"type": "bar"}, {"type": "scatter"}],
    #            [{"type": "bar"}, {"type": "scatter"}]]
    # )
    
    # TODO: Add traces for each subplot
    # Chart 1: Top states (row=1, col=1)
    # Chart 2: Satisfaction trend (row=1, col=2)
    # Chart 3: Top categories (row=2, col=1)
    # Chart 4: Category scatter (row=2, col=2)
    
    # TODO: Update layout with proper sizing and titles
    # fig.update_layout(
    #     height=800,
    #     title_text="Olist E-commerce Executive Dashboard",
    #     title_x=0.5,
    #     showlegend=False
    # )
    
    # fig.show()
    pass

## 📊 Reflection and Analysis

Complete the following reflection on your interactive visualizations:

### Business Insights Summary

**Based on your interactive visualizations, answer the following:**

1. **Geographic Analysis**: Which Brazilian state generates the most revenue, and what percentage of total revenue does it represent?
   
   *Your answer here:*

2. **Customer Satisfaction**: What trend do you observe in customer satisfaction over time? Are there any concerning patterns?
   
   *Your answer here:*

3. **Product Categories**: Which product category offers the best combination of high revenue and high customer satisfaction?
   
   *Your answer here:*

4. **Interactive Advantages**: List three specific advantages your interactive charts provide over static versions:
   
   - *Advantage 1:*
   - *Advantage 2:*
   - *Advantage 3:*

### Technical Reflection

5. **Plotly Features**: Which Plotly interactive features did you find most valuable for business analysis?
   
   *Your answer here:*

6. **Dashboard Design**: How does combining multiple charts in a dashboard improve the storytelling compared to individual charts?
   
   *Your answer here:*

In [ ]:
# Clean up database connection
if db_engine:
    db_engine.dispose()
    print("🔒 Database connection closed")

print("\n✅ Assignment Complete!")
print(f"📝 Student: {STUDENT_INFO['name']}")
print(f"📅 Submission Date: {STUDENT_INFO['submission_date']}")
print("\n📊 Summary of Completed Tasks:")
print("  ✅ Task 1: Interactive geographic visualization")
print("  ✅ Task 2: Customer satisfaction time series")
print("  ✅ Task 3: Product category performance scatter plot")
print("  ✅ Task 4: Executive dashboard with multiple charts")
print("  ✅ Business insights and reflection")

## 📋 Submission Checklist

Before submitting, ensure you have completed:

- [ ] **Student Information**: Updated name and student ID at the top
- [ ] **Database Connection**: Successfully connected to Supabase
- [ ] **Task 1**: Created interactive geographic sales visualization
- [ ] **Task 2**: Built customer satisfaction time series with interactivity
- [ ] **Task 3**: Developed product category performance scatter plot
- [ ] **Task 4**: Combined visualizations into executive dashboard
- [ ] **Business Insights**: Completed reflection and analysis section
- [ ] **Code Quality**: All cells execute without errors
- [ ] **Professional Styling**: Charts are well-formatted and business-appropriate

## 🎯 Grading Rubric (20 Points Total)

| Component | Excellent (4-5) | Good (3) | Satisfactory (2) | Needs Improvement (0-1) |
|-----------|----------------|----------|------------------|------------------------|
| **Task 1: Geographic Viz** | Interactive, professional, insightful | Good interactivity, clear | Basic conversion, limited features | Non-functional or static |
| **Task 2: Time Series** | Advanced features, dual axes, professional | Good interactivity, clear trends | Basic time series, limited features | Poor implementation |
| **Task 3: Scatter Plot** | Multi-dimensional, hover details, insights | Good scatter plot, some features | Basic scatter, limited interactivity | Poor visualization |
| **Task 4: Dashboard** | Professional layout, cohesive design | Good integration, clear layout | Basic dashboard, adequate design | Poor integration |
| **Business Insights** | Deep analysis, actionable insights | Good insights, clear thinking | Basic analysis, surface insights | Minimal or unclear analysis |

**Total: ___/20 Points**

---

**🎉 Congratulations on completing your interactive visualization assignment! You've successfully bridged static analysis with dynamic, engaging visualizations that tell compelling business stories.**